In [1]:
# import numpy as np
import cvxpy as cp
import networkx as nx
import autograd.numpy as np
from autograd import grad

import pymanopt
import pymanopt.manifolds
import pymanopt.solvers

from numpy.random import default_rng
from complex_elliptope import ComplexElliptope

In [2]:
def decompose_psd(X):
    n = X.shape[0]
    eigen_values, eigen_vectors = np.linalg.eigh(X)
    non_neg_eig_idx = [i for i in range(n) if eigen_values[i] >= 0]
    return eigen_vectors[:, non_neg_eig_idx] @ np.diag(np.sqrt(eigen_values[non_neg_eig_idx]))

In [3]:
def normalize_rows(X):
    # normalize the rows of matrix X
    return X / np.linalg.norm(X, axis=1)[:, np.newaxis]

# Phase recovery

In [4]:
n = 20  # number of observations
p = 5  # dimension of x
max_val = 1e2

In [5]:
rng = default_rng()
A = rng.random((n,p)) * max_val + rng.random((n,p)) * max_val * 1j
assert np.linalg.matrix_rank(A, tol=1e-6) >= p  # A must be injective; if the rows of A are linearly independent, AA+ = I
b = rng.random(n) * max_val
M = np.diag(b) @ (np.identity(n) - A @ np.linalg.pinv(A)) @ np.diag(b)

\begin{equation*}
\begin{aligned}
\min \quad & u^{*}Mu \\
\textrm{s.t.} \quad & |u_{i}| = 1, \qquad i=1,...,n
\end{aligned}
\end{equation*}

In [6]:
# u = cp.Variable(n, complex=True)
# qp_constraints = [ u[i].value.H * u[i] == 1 for i in range(n) ]
# prob = cp.Problem(cp.Minimize(cp.quad_form(u, M)), qp_constraints)
# prob.solve()

\begin{equation*}
\begin{aligned}
\min \quad & M \cdot U \\
\textrm{s.t.} \quad & U_{ii} = 1, \qquad i=1,...,n \\
& U \succeq 0
\end{aligned}
\end{equation*}

### solve SDP with cvxpy

In [7]:
U = cp.Variable((n,n), hermitian=True)

In [8]:
sdp_constraints = [U >> 0]
sdp_constraints += [U[i][i] == 1 for i in range(n)]

In [9]:
prob = cp.Problem(cp.Minimize(cp.real(cp.trace(M @ U))), sdp_constraints)
prob.solve()

print("The optimal value is", prob.value)
print("A solution U is", U.value)

The optimal value is 3757.273336011314
A solution U is [[ 1.        +0.j          0.13563735-0.02972601j  0.18216343-0.60187j
   0.31493782-0.14971841j  0.48059807+0.30853365j  0.62118225-0.33683035j
   0.16319677-0.02450065j  0.47936591-0.12748726j -0.18572936+0.28474695j
   0.32461147+0.53771399j  0.55403705-0.46552805j  0.51169474-0.57187825j
  -0.24496001+0.71684129j -0.60317887+0.59996214j  0.7470481 -0.24824028j
  -0.15168217+0.47805492j  0.70514488+0.43173832j  0.44518285+0.0947539j
   0.13204699+0.22211239j  0.56923396-0.33485159j]
 [ 0.13563735+0.02972601j  1.        +0.j         -0.50803035+0.46203458j
  -0.64872916-0.62510302j  0.65892632-0.48915292j -0.42046199-0.50269725j
   0.38433819-0.90585662j  0.67579954-0.61212486j  0.64452601-0.60520889j
   0.77803751+0.25958146j -0.234446  -0.64878159j -0.41224365+0.33076696j
   0.58991869+0.03898952j  0.42079248+0.06948571j -0.21060637-0.53204285j
   0.7524681 +0.39835448j  0.02272024-0.47424963j  0.285465  -0.82574153j
   0.00636

In [10]:
opt_rank = np.linalg.matrix_rank(U.value,1e-9)
print(opt_rank)

2


### Complex elliptope gradient

In [11]:
manifold = ComplexElliptope(n, opt_rank)

@pymanopt.function.Autograd(manifold)
def manifold_cost(Y):
    return np.real(np.trace(M @ Y @ np.conj(Y).T))

problem = pymanopt.Problem(manifold=manifold, cost=manifold_cost)
solver = pymanopt.solvers.SteepestDescent(minstepsize=1e-6)
solution = solver.solve(problem)

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         +2.7260360487467751e+04    5.17153161e+03    
   2         +2.3240681880814522e+04    3.28200245e+03    
   3         +2.3124024040173874e+04    6.02401381e+03    
   4         +2.2665556129563738e+04    5.81686891e+03    
   5         +2.0989809108241388e+04    4.87739685e+03    
   6         +1.7664461692389596e+04    2.45766418e+03    
   7         +1.7208259154803440e+04    5.08246793e+03    
   8         +1.5668679594617079e+04    3.56426896e+03    
   9         +1.5251739925163445e+04    3.84701479e+03    
  10         +1.4027418394700049e+04    1.89497579e+03    
  11         +1.3689270682229773e+04    1.72795970e+03    
  12         +1.3612608288956555e+04    2.37736949e+03    
  13         +1.3344936116564848e+04    1.78976548e+03    
  14         +1.3047031011195319e+04    1.67427182e+03    
  15         +1.2883654489267206e+04    2.

In [12]:
Y = manifold.rand()

### Hyperplane rounding

In [ ]:
def complex_hyperplane_rounding(Y, cost, iter=100):
    min_cost = np.Inf
    best_x = None
    d = Y.shape[1]
    rng = default_rng()
    for i in range(iter):
        # r = rng.random((d,1)) + rng.random((d,1)) * 1j
        r = rng.normal(size=(d,1)) + rng.normal(size=(d,1)) * 1j
        x = normalize_rows(Y @ r)
        cost_val = cost(x)
        if cost_val < min_cost:
            min_cost = cost_val
            best_x = x
    return best_x, min_cost

In [75]:
qp_cost = lambda u: cp.real(cp.quad_form(u, M)).value

In [76]:
complex_hyperplane_rounding(decompose_psd(U.value), qp_cost)

(array([[ 0.6859097 -0.72768666j],
        [ 0.96647216-0.25677142j],
        [-0.82016485-0.57212727j],
        [ 0.7409845 +0.67152212j],
        [ 0.18079908-0.98352005j],
        [-0.7675359 -0.64100596j],
        [-0.01098831-0.99993963j],
        [ 0.31173664+0.95016855j],
        [ 0.99715064-0.07543605j],
        [ 0.3746287 +0.92717492j],
        [ 0.27474116-0.96151823j],
        [-0.59991379+0.80006465j],
        [ 0.52678665+0.84999755j],
        [ 0.239056  +0.97100578j],
        [ 0.21197388+0.97727533j],
        [-0.30207097+0.95328544j],
        [ 0.75793075+0.65233502j],
        [-0.91205091+0.410077j  ],
        [-0.73911797-0.673576j  ],
        [-0.96335864+0.26821658j]]),
 array([[4052.57355489]]))

# Max-cut (for reference)

In [77]:
data_path = "../dat/"
# graph_file = "torusg3-8.dat"
graph_file = "toruspm3-8-50.dat"

with open(data_path + graph_file) as inf:
    next(inf, '')   # skip first line
    G = nx.read_weighted_edgelist(inf, nodetype=int, encoding="utf-8")

In [78]:
first_vertex = np.floor(default_rng().random() * (len(G) - n - 1)).astype(int)
G = G.subgraph(range(first_vertex, first_vertex + n))
L = nx.laplacian_matrix(G).toarray() * 1.0

In [79]:
X = cp.Variable((n,n), PSD=True)

\begin{equation*}
\begin{aligned}
\max \quad & \frac{1}{4} L \cdot X \\
\textrm{s.t.} \quad & X_{ii} = 1, \qquad i=1,...,n \\
& X \succeq 0
\end{aligned}
\end{equation*}

In [80]:
constraints = [ X[i][i] == 1 for i in range(n) ]

In [81]:
prob = cp.Problem(cp.Maximize(1/4 * (cp.trace(L @ X))), constraints)
prob.solve()

14.357087913371045

### Hyperplane rounding

In [82]:
def hyperplane_rounding(Y, cost, iter=100):
    min_cost = 0
    best_x = None
    d = Y.shape[1]
    rng = default_rng()
    for i in range(iter):
        r = rng.random((d,1))
        r = r / np.linalg.norm(r)
        x = np.sign(Y @ r)
        cost_val = cost(x)
        if cost_val < min_cost:
            min_cost = cost_val
            best_x = x
    return best_x, min_cost

In [83]:
mc_cost = lambda x: -1/4 * (cp.quad_form(x, L)).value

In [84]:
hyperplane_rounding(decompose_psd(X.value), mc_cost)

(array([[-1.],
        [ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.]]),
 array([[-14.]]))